# Latent Dirichlet Allocation

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [34]:
import pandas as pd

In [35]:
npr = pd.read_csv('npr.csv')

In [78]:
npr['Article']

0        In the Washington of 2016, even when the polic...
1          Donald Trump has used Twitter  —   his prefe...
2          Donald Trump is unabashedly praising Russian...
3        Updated at 2:50 p. m. ET, Russian President Vl...
4        From photography, illustration and video, to d...
                               ...                        
11987    The number of law enforcement officers shot an...
11988      Trump is busy these days with victory tours,...
11989    It’s always interesting for the Goats and Soda...
11990    The election of Donald Trump was a surprise to...
11991    Voters in the English city of Sunderland did s...
Name: Article, Length: 11992, dtype: object

In [36]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [38]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [39]:
dtm = cv.fit_transform(npr['Article'])

In [40]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## LDA

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [21]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words

In [42]:
feature_names = cv.get_feature_names_out()

In [43]:
len(feature_names)

54777

In [44]:
import random

In [72]:
for i in range(20):
    random_word_id = random.randint(0,54776)
    print(feature_names[random_word_id])

aspen
redlined
flickers
estrella
resourceful
muddying
outsource
inventive
backtracking
ursa
ballyhooed
electrolyte
yuanhui
neonics
republics
touma
disapproved
elect
824
kym


In [73]:
for i in range(20):
    random_word_id = random.randint(0,54776)
    print(feature_names[random_word_id])

keytar
sferrazza
desperate
epidural
cello
categories
reconfigure
repairman
manifold
plaintively
scaring
calcifications
unsafe
liabilities
686
morada
wastewater
pogue
entrenched
vigilante


### Showing Top Words Per Topic

In [48]:
len(LDA.components_)

7

In [49]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [75]:
len(LDA.components_[0])

54777

In [92]:
single_topic = LDA.components_[1]

In [93]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 7793, 17024,  1089, ..., 38079, 50426, 42561])

In [94]:
# Word least representative of this topic
single_topic[18302]

0.14286154754910452

In [95]:
# Word most representative of this topic
single_topic[42993]

2611.376403764393

In [96]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33992, 40955, 42993, 33419, 36283, 49613, 37374, 38079, 50426,
       42561])

In [97]:
top_word_indices = single_topic.argsort()[-10:]

In [98]:
for index in top_word_indices:
    print(feature_names[index])

npr
reports
says
news
people
told
police
president
trump
said


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [99]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([feature_names[i] for i in topic.argsort()[-20:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['president', 'state', 'tax', 'insurance', 'trump', 'companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['white', 'according', 'attack', 'reported', 'war', 'military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['little', 'know', 'don', 'year', 'make', 'way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['world', 'research', 'university', 'percent', 'care', 'time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['donald', 'political', 'states', 'law', 'just', 'voters', 'vote', 'election', 'par

### Attaching Discovered Topic Labels to Original Articles

In [60]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [61]:
dtm.shape

(11992, 54777)

In [62]:
len(npr)

11992

In [63]:
topic_results = LDA.transform(dtm)

In [100]:
topic_results

array([[1.61040465e-02, 6.83341493e-01, 2.25376318e-04, ...,
        2.99652737e-01, 2.25479379e-04, 2.25497980e-04],
       [3.63424997e-02, 8.86130697e-01, 4.40751747e-04, ...,
        7.57636804e-02, 4.40866779e-04, 4.40835574e-04],
       [3.28569485e-04, 6.96344889e-01, 3.28302105e-04, ...,
        3.02012902e-01, 3.28724083e-04, 3.28352652e-04],
       ...,
       [1.44467964e-02, 1.60696622e-01, 1.73678310e-01, ...,
        2.24636569e-02, 3.98728349e-04, 3.98359730e-04],
       [4.33560738e-04, 3.53196803e-02, 4.33022554e-04, ...,
        9.62512640e-01, 4.33971991e-04, 4.33490254e-04],
       [3.98777533e-01, 2.54376049e-04, 3.59290659e-01, ...,
        2.40914375e-01, 2.54445555e-04, 2.54253739e-04]])

In [64]:
topic_results.shape

(11992, 7)

In [65]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [66]:
topic_results[0].round(2)

array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [102]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [68]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

In [69]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0])

In [70]:
npr['Topic'] = topic_results.argmax(axis=1)

In [71]:
npr.head(10)

Article  Topic
0  In the Washington of 2016, even when the polic...      1
1    Donald Trump has used Twitter  —   his prefe...      1
2    Donald Trump is unabashedly praising Russian...      1
3  Updated at 2:50 p. m. ET, Russian President Vl...      1
4  From photography, illustration and video, to d...      2
5  I did not want to join yoga class. I hated tho...      3
6  With a   who has publicly supported the debunk...      3
7  I was standing by the airport exit, debating w...      2
8  If movies were trying to be more realistic, pe...      3
9  Eighteen years ago, on New Year’s Eve, David F...      2

## Great work!